In [95]:
# Алгоритм считает 100 возможных стратегий за 21.3 сек
# 1000 комбинаций - за 120 секунд
# 25344 - 45 минут 

In [96]:
# shell - до 12/30/2022
# exxonmobil - до 12/30/2022
# newport - до 12/30/2022
# bp - до 12/30/2022
# barrick - до 12/30/2022
# chinapetroleum - до 12.02.2022

In [97]:
# !pip install yahoo_fin
# !pip install yoptions
# !pip install pandas_datareader
# pip install lightgbm
# !pip install --upgrade xlrd

In [98]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{:.2f}'.format

import datetime as dt
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.pyplot import figure
import seaborn as sns
import os
import datetime
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [99]:
CFTC_futures_only = pd.read_csv('C:/Users/LT17/Desktop/Другое/Учёба/Все файлы от 30.12.22/CFTC_futures_only.csv')
all_commodities = pd.read_csv('C:/Users/LT17/Desktop/Другое/Учёба/Все файлы от 30.12.22/all_commodities_modified.csv') 


new_gas = pd.read_csv('C:/Users/LT17/Desktop/Другое/Учёба/Все файлы от 30.12.22/Новыекоммодитис/Natural Gas Futures Historical Data.csv')
new_oil = pd.read_csv('C:/Users/LT17/Desktop/Другое/Учёба/Все файлы от 30.12.22/Новыекоммодитис/Brent Oil Futures Historical Data.csv')
new_gold = pd.read_csv('C:/Users/LT17/Desktop/Другое/Учёба/Все файлы от 30.12.22/Новыекоммодитис/Gold Futures Historical Data.csv')
new_gas['ticker'] = 'Natural_Gas_Futures_new'
new_oil['ticker'] = 'CRUDE_OIL_new'
new_gold['ticker'] = 'Gold_Futures_new'

all_commodities = pd.concat([all_commodities, new_gas], axis = 0, ignore_index=False)
all_commodities = pd.concat([all_commodities, new_oil], axis = 0, ignore_index=False)
all_commodities = pd.concat([all_commodities, new_gold], axis = 0, ignore_index=False)

all_commodities.drop_duplicates(subset = all_commodities.columns, inplace = True)
CFTC_futures_only.drop_duplicates(subset = CFTC_futures_only.columns, inplace = True)

CFTC_futures_only.Report_Date_as_MM_DD_YYYY = pd.to_datetime(CFTC_futures_only.Report_Date_as_MM_DD_YYYY)
all_commodities.Date = pd.to_datetime(all_commodities.Date)
CFTC_futures_only.sort_values(by = 'Report_Date_as_MM_DD_YYYY', ascending = False, inplace = True)
all_commodities.drop(columns = ['Unnamed: 0'], inplace = True)

# Выравниваем датафреймы CFTC и с Коммодитис
all_commodities = all_commodities.loc[all_commodities.Date >= CFTC_futures_only.Report_Date_as_MM_DD_YYYY.min()]

# (!!!) Начиная с февраля 2022 года ряд фьючерсов по natural газ изменили наименования _____________________________
# CFTC_futures_only[CFTC_futures_only.Market_and_Exchange_Names.str.contains('GAS')].Market_and_Exchange_Names.unique()

CFTC_futures_only.Market_and_Exchange_Names = np.where(CFTC_futures_only.Market_and_Exchange_Names.str \
                                                       .contains('NAT GAS NYME - NEW YORK MERCANTILE EXCHANGE'),
                                                       str('NATURAL GAS - NEW YORK MERCANTILE EXCHANGE'),
                                                       CFTC_futures_only.Market_and_Exchange_Names)

# NATURAL GAS - NEW YORK MERCANTILE EXCHANGE был переименован в NAT GAS NYME - NEW YORK MERCANTILE EXCHANGE
# NAT GAS NYME - NEW YORK MERCANTILE EXCHANGE будем использовать для дальнейшего анализа

In [100]:
# for i in CFTC_futures_only[CFTC_futures_only.Market_and_Exchange_Names.str.contains('OIL')].Market_and_Exchange_Names.unique():
#     if CFTC_futures_only[CFTC_futures_only.Market_and_Exchange_Names.str.contains(i)].Report_Date_as_MM_DD_YYYY.max() > pd.to_datetime("2022-12-01"):
#         print(i, '//', CFTC_futures_only[CFTC_futures_only.Market_and_Exchange_Names.str.contains(i)].Report_Date_as_MM_DD_YYYY.max(), '//min: ', CFTC_futures_only[CFTC_futures_only.Market_and_Exchange_Names.str.contains(i)].Report_Date_as_MM_DD_YYYY.min())

In [101]:
all_commodities = all_commodities.applymap(lambda x: str(x).replace(',', ''))

all_commodities.Date = pd.to_datetime(all_commodities.Date)
#
all_commodities.Price = all_commodities.Price.astype('float')
all_commodities.Open = all_commodities.Open.astype('float')
all_commodities.High = all_commodities.High.astype('float')
all_commodities.Low = all_commodities.Low.astype('float')
#
all_commodities.rename(columns = {'Vol.': 'Volume',
                               'Change %': 'growth_rate'}, inplace = True)
all_commodities.Volume.replace('nan', 0, inplace = True)
all_commodities.growth_rate = all_commodities.growth_rate.apply(lambda x: str(x).replace('%', ''))
all_commodities.ticker = all_commodities.ticker.apply(lambda x: str(x).replace('HistoricalData', '')) ####
#
all_commodities.Date = pd.to_datetime(all_commodities.Date) ####
all_commodities['year'] = all_commodities.Date.dt.year

In [102]:
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 10
        return 1000.0
    
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 10000
        return 1000000.0
    
    if 'B' in x:
        return float(x.replace('B', '')) * 10000000
    return 0.0

all_commodities.growth_rate = all_commodities.growth_rate.apply(lambda x: float(x))
all_commodities.Volume = all_commodities.Volume.apply(value_to_float)

# FUTURES
Oil = all_commodities.query('ticker == "CRUDE_OIL_new" & year >= 2006')
Gas = all_commodities.query('ticker == "Natural_Gas_Futures_new" & year >= 2006')
gold = all_commodities.query('ticker == "Gold_Futures_new" & year >= 2006')


# STOCKS
shell = pd.read_csv('C:/Users/LT17/Desktop/Другое/Учёба/Все файлы от 30.12.22/Фондовый рынок/SHELL_oil.csv') 
exxon = pd.read_csv('C:/Users/LT17/Desktop/Другое/Учёба/Все файлы от 30.12.22/Фондовый рынок/exxon_oil.csv')
barrickgold = pd.read_csv('C:/Users/LT17/Desktop/Другое/Учёба/Все файлы от 30.12.22/Фондовый рынок/barrickgold.csv')
newportgold = pd.read_csv('C:/Users/LT17/Desktop/Другое/Учёба/Все файлы от 30.12.22/Фондовый рынок/newport_gold.csv')
BP = pd.read_csv('C:/Users/LT17/Desktop/Другое/Учёба/Все файлы от 30.12.22/Фондовый рынок/BP.csv')
petrochina = pd.read_csv('C:/Users/LT17/Desktop/Другое/Учёба/Все файлы от 30.12.22/Фондовый рынок/chinapetroleum.csv')

shell.rename(columns = {'Vol.': 'Volume',
                               'Изм. %': 'growth_rate'}, inplace = True)
exxon.columns = list(shell.columns)
barrickgold.columns = list(shell.columns)
newportgold.columns = list(shell.columns)
BP.columns = list(shell.columns)
petrochina.columns = list(shell.columns)

barrickgold.Date = pd.to_datetime(barrickgold.Date).dt.strftime('%Y-%m-%d')
newportgold.Date = pd.to_datetime(newportgold.Date).dt.strftime('%Y-%m-%d')
BP.Date = pd.to_datetime(BP.Date).dt.strftime('%Y-%m-%d')
petrochina.Date = pd.to_datetime(petrochina.Date).dt.strftime('%Y-%m-%d')

shell[['Date', 'Price', 'Open', 'High', 'Low', 'Change %']] = shell[['Date',
                                                                     'Price',
                                                                     'Open',
                                                                     'High',
                                                                     'Low',
                                                                     'Change %']] \
                                                              .applymap(lambda x: str(x).replace(',', '.'))

exxon[['Date', 'Price', 'Open', 'High', 'Low', 'Change %']] = exxon[['Date',
                                                                     'Price',
                                                                     'Open',
                                                                     'High',
                                                                     'Low',
                                                                     'Change %']] \
                                                              .applymap(lambda x: str(x).replace(',', '.'))

barrickgold[['Date', 'Price', 'Open', 'High', 'Low', 'Change %']] = barrickgold[['Date',
                                                                                 'Price',
                                                                                 'Open',
                                                                                 'High',
                                                                                 'Low',
                                                                                 'Change %']] \
                                                              .applymap(lambda x: str(x).replace(',', '.'))

newportgold[['Date', 'Price', 'Open', 'High', 'Low', 'Change %']] = newportgold[['Date',
                                                                                 'Price',
                                                                                 'Open',
                                                                                 'High',
                                                                                 'Low',
                                                                                 'Change %']] \
                                                              .applymap(lambda x: str(x).replace(',', '.'))

BP[['Date', 'Price', 'Open', 'High', 'Low', 'Change %']] = BP[['Date',
                                                                     'Price',
                                                                     'Open',
                                                                     'High',
                                                                     'Low',
                                                                     'Change %']] \
                                                              .applymap(lambda x: str(x).replace(',', '.'))

petrochina[['Date', 'Price', 'Open', 'High', 'Low', 'Change %']] = petrochina[['Date',
                                                                               'Price',
                                                                               'Open',
                                                                               'High',
                                                                               'Low',
                                                                               'Change %']] \
                                                              .applymap(lambda x: str(x).replace(',', '.'))


shell.Volume = shell.Volume.apply(lambda x: str(x).replace(',', ''))
exxon.Volume = exxon.Volume.apply(lambda x: str(x).replace(',', ''))
barrickgold.Volume = barrickgold.Volume.apply(lambda x: str(x).replace(',', ''))
newportgold.Volume = newportgold.Volume.apply(lambda x: str(x).replace(',', ''))
BP.Volume = BP.Volume.apply(lambda x: str(x).replace(',', ''))
petrochina.Volume = petrochina.Volume.apply(lambda x: str(x).replace(',', ''))

newportgold.Volume = newportgold.Volume.apply(value_to_float).astype('int')
barrickgold.Volume = barrickgold.Volume.apply(value_to_float).astype('int')
BP.Volume = BP.Volume.apply(value_to_float).astype('int')
petrochina.Volume = petrochina.Volume.apply(value_to_float).astype('int')

newportgold.Date = pd.to_datetime(newportgold.Date, dayfirst=False)
barrickgold.Date = pd.to_datetime(barrickgold.Date, dayfirst=False)
exxon.Date = pd.to_datetime(exxon.Date, dayfirst=False)
shell.Date = pd.to_datetime(shell.Date, dayfirst=False)
BP.Date = pd.to_datetime(BP.Date, dayfirst=False)
petrochina.Date = pd.to_datetime(petrochina.Date, dayfirst=False)

# exxon = exxon.loc[(exxon.Date >= CFTC_futures_only.Report_Date_as_MM_DD_YYYY.min()) & (exxon.Date <= Oil.Date.max())]
# shell = shell.loc[(shell.Date >= CFTC_futures_only.Report_Date_as_MM_DD_YYYY.min()) & (shell.Date <= Oil.Date.max())]
# BP = BP.loc[(BP.Date >= CFTC_futures_only.Report_Date_as_MM_DD_YYYY.min()) & (BP.Date <= Oil.Date.max())]
# petrochina = petrochina.loc[(petrochina.Date >= CFTC_futures_only.Report_Date_as_MM_DD_YYYY.min()) & (petrochina.Date <= Oil.Date.max())]

# shell = shell.loc[shell.Date.isin(list(exxon.Date))]
# exxon = exxon.loc[exxon.Date.isin(list(shell.Date))]

# BP = BP.loc[BP.Date.isin(list(exxon.Date))]
# petrochina = petrochina.loc[petrochina.Date.isin(list(shell.Date))]

shell = shell.query('Date >= "2021-06-15"')
exxon = exxon.query('Date >= "2021-06-15"')
BP = BP.query('Date >= "2021-06-15"')
petrochina = petrochina.query('Date >= "2021-06-15"')
barrickgold = barrickgold.query('Date >= "2021-06-15"')
newportgold = newportgold.query('Date >= "2021-06-15"')

Oil = Oil.loc[Oil.Date.isin(list(shell.Date))]
Gas = Gas.loc[Gas.Date.isin(list(shell.Date))]
gold = gold.loc[gold.Date.isin(list(shell.Date))]

# newportgold = newportgold.loc[(newportgold.Date >= CFTC_futures_only.Report_Date_as_MM_DD_YYYY.min()) & (newportgold.Date <= gold.Date.max())]
# barrickgold = barrickgold.loc[(barrickgold.Date >= CFTC_futures_only.Report_Date_as_MM_DD_YYYY.min()) & (barrickgold.Date <= gold.Date.max())]
# gold = gold.loc[gold.Date.isin(list(barrickgold.Date))]

# 2021-06-15


# exxon.Price = exxon.Price.apply(lambda x: float(x))
# shell.Price = shell.Price.apply(lambda x: float(x))
# barrickgold.Price = barrickgold.Price.apply(lambda x: float(x))
# newportgold.Price = newportgold.Price.apply(lambda x: float(x))

# print('Newport gold: Размер выборки: ', newportgold.shape)
# print('Barrick gold: Размер выборки: ', barrickgold.shape,'\n')
# print('Exxon: Размер выборки: ', exxon.shape)
# print('BP: Размер выборки: ', BP.shape)
# print('Petrochina: Размер выборки: ', petrochina.shape)
# print('Shell: Размер выборки: ', shell.shape, '\n')
# print('Gas: Размер выборки: ', Gas.shape)
# print('Oil: Размер выборки: ', Oil.shape,'\n')

## Форматируем датафреймы с динамикой базового актива фьючерсов, с динамикой фьючерсов из отчета CFTC, с акциями "зеркальных" активов

In [103]:
# АКЦИИ__________________________________________________________________________________________________________

# SHELL (OIL / GAS)
shell[['High', 'Low']] = shell[['High', 'Low']].applymap(lambda x: float(x))
shell['intraday_diff'] = shell.High - shell.Low
shell.Volume = shell.Volume.apply(value_to_float)
shell.sort_values(by = 'Date', ascending = True, inplace = True)
shell['2weeks_savg'] = shell['Price'].rolling(14).mean()
shell = shell[['Date', 'Price', 'Volume', 'intraday_diff', '2weeks_savg']]
shell.dropna(inplace = True)

# Petrochina (OIL/ GAS)
petrochina[['High', 'Low']] = petrochina[['High', 'Low']].applymap(lambda x: float(x))
petrochina['intraday_diff'] = petrochina.High - petrochina.Low
petrochina.Volume = petrochina.Volume.apply(value_to_float)
petrochina.sort_values(by = 'Date', ascending = True, inplace = True)
petrochina['2weeks_savg'] = petrochina['Price'].rolling(14).mean()
petrochina = petrochina[['Date', 'Price', 'Volume', 'intraday_diff', '2weeks_savg']]
petrochina.dropna(inplace = True)

# BP (OIL/ GAS)
exxon[['High', 'Low']] = exxon[['High', 'Low']].applymap(lambda x: float(x))
exxon['intraday_diff'] = exxon.High - exxon.Low
exxon.Volume = exxon.Volume.apply(value_to_float)
exxon.sort_values(by = 'Date', ascending = True, inplace = True)
exxon['2weeks_savg'] = exxon['Price'].rolling(14).mean()
exxon = exxon[['Date', 'Price', 'Volume', 'intraday_diff', '2weeks_savg']]
exxon.dropna(inplace = True)

# ExxonMobil (OIL/ GAS)
BP[['High', 'Low']] = BP[['High', 'Low']].applymap(lambda x: float(x))
BP['intraday_diff'] = BP.High - BP.Low
BP.Volume = BP.Volume.apply(value_to_float)
BP.sort_values(by = 'Date', ascending = True, inplace = True)
BP['2weeks_savg'] = BP['Price'].rolling(14).mean()
BP = BP[['Date', 'Price', 'Volume', 'intraday_diff', '2weeks_savg']]
BP.dropna(inplace = True)

# newportgold (GOLD)
newportgold[['High', 'Low']] = newportgold[['High', 'Low']].applymap(lambda x: float(x))
newportgold['intraday_diff'] = newportgold.High - newportgold.Low
newportgold.Volume = newportgold.Volume.apply(value_to_float)
newportgold.sort_values(by = 'Date', ascending = True, inplace = True)
newportgold['2weeks_savg'] = newportgold['Price'].rolling(14).mean()
newportgold = newportgold[['Date', 'Price', 'Volume', 'intraday_diff', '2weeks_savg']]
newportgold.dropna(inplace = True)

# barrickgold (GOLD)
barrickgold[['High', 'Low']] = barrickgold[['High', 'Low']].applymap(lambda x: float(x))
barrickgold['intraday_diff'] = barrickgold.High - barrickgold.Low
barrickgold.Volume = barrickgold.Volume.apply(value_to_float)
barrickgold.sort_values(by = 'Date', ascending = True, inplace = True)
barrickgold['2weeks_savg'] = barrickgold['Price'].rolling(14).mean()
barrickgold = barrickgold[['Date', 'Price', 'Volume', 'intraday_diff', '2weeks_savg']]
barrickgold.dropna(inplace = True)

# ФЬЮЧЕРСЫ ______________________________________________________________________________________________________

Oil['intraday_diff'] = Oil.High - Oil.Low
Oil.rename(columns = {'ticker': 'Futures_Oil'}, inplace = True)

Oil = Oil[['Date',
           'Price',
           'intraday_diff',
           'growth_rate',
           'Volume',
           'Futures_Oil']]

gold['intraday_diff'] = gold.High - gold.Low
gold.rename(columns = {'ticker': 'Futures_gold'}, inplace = True)

gold = gold[['Date',
           'Price',
           'intraday_diff',
           'growth_rate',
           'Volume',
           'Futures_gold']]

# CFTC data _____________________________________________________________________________________________________

cftc_gold = CFTC_futures_only.query('Market_and_Exchange_Names == "GOLD - COMMODITY EXCHANGE INC."')

cftc_gold = cftc_gold[
                  ['Report_Date_as_MM_DD_YYYY',
                   'Open_Interest_All',
                   'Tot_Rept_Positions_Long_All',
                   'Tot_Rept_Positions_Short_All']
                   ]

cftc_gold['difference'] = cftc_gold.Tot_Rept_Positions_Long_All - cftc_gold.Tot_Rept_Positions_Short_All
cftc_gold.sort_values(by = 'Report_Date_as_MM_DD_YYYY', ascending = True, inplace = True )
cftc_gold['rolling_52_min'] = cftc_gold.difference.rolling(52).min()
cftc_gold['rolling_52_max'] = cftc_gold.difference.rolling(52).max()
cftc_gold['Williams_coef'] = round((cftc_gold.difference - cftc_gold['rolling_52_min']) / (cftc_gold['rolling_52_max'] - cftc_gold['rolling_52_min'])*100,2)
cftc_gold.dropna(inplace = True)

n_steps = 7
def my_fun(x):
    return x.iloc[-1] - x.iloc[0]

# # Раcчет индикатора Стивена Бриза (Movement Index) (Индекс Ульямса за текущую неделю за вычетом Индекса Ульямса N недель назад)
# # Если Movement Index > 40%, это означает завершение коррекции долгосрочного тренда; 
# # Если Movement Index < -40%, это означает завершение долгосрочного тренда;
cftc_gold['Breeze_coef'] = cftc_gold['Williams_coef'].rolling(window=n_steps).apply(my_fun)

cftc_gold = cftc_gold[['Report_Date_as_MM_DD_YYYY',
                       'Open_Interest_All',
                       'Tot_Rept_Positions_Long_All', 
                       'Tot_Rept_Positions_Short_All',
                       'difference',
                       'rolling_52_min',
                       'rolling_52_max',
                       'Williams_coef',
                       'Breeze_coef']]
# _______________________________________________________________________________________________________________

cftc_oil = CFTC_futures_only.query('Market_and_Exchange_Names == "CRUDE OIL, LIGHT SWEET-WTI - ICE FUTURES EUROPE"')

cftc_oil = cftc_oil[
                  ['Report_Date_as_MM_DD_YYYY',
                   'Open_Interest_All',
                   'Tot_Rept_Positions_Long_All',
                   'Tot_Rept_Positions_Short_All']
                   ]

cftc_oil['difference'] = cftc_oil.Tot_Rept_Positions_Long_All - cftc_oil.Tot_Rept_Positions_Short_All
cftc_oil.sort_values(by = 'Report_Date_as_MM_DD_YYYY', ascending = True, inplace = True )
cftc_oil['rolling_52_min'] = cftc_oil.difference.rolling(52).min()
cftc_oil['rolling_52_max'] = cftc_oil.difference.rolling(52).max()
cftc_oil['Williams_coef'] = round((cftc_oil.difference - cftc_oil['rolling_52_min']) / (cftc_oil['rolling_52_max'] - cftc_oil['rolling_52_min'])*100,2)
cftc_oil['Breeze_coef'] = cftc_oil['Williams_coef'].rolling(window=n_steps).apply(my_fun)
cftc_oil.dropna(inplace = True)

cftc_oil = cftc_oil[['Report_Date_as_MM_DD_YYYY',
                       'Open_Interest_All',
                       'Tot_Rept_Positions_Long_All', 
                       'Tot_Rept_Positions_Short_All',
                       'difference',
                       'rolling_52_min',
                       'rolling_52_max',
                       'Williams_coef',
                       'Breeze_coef']]
# _______________________________________________________________________________________________________________

cftc_gas = CFTC_futures_only.query('Market_and_Exchange_Names == "NATURAL GAS - NEW YORK MERCANTILE EXCHANGE"')

cftc_gas = cftc_gas[
                  ['Report_Date_as_MM_DD_YYYY',
                   'Open_Interest_All',
                   'Tot_Rept_Positions_Long_All',
                   'Tot_Rept_Positions_Short_All']
                   ]

cftc_gas['difference'] = cftc_gas.Tot_Rept_Positions_Long_All - cftc_gas.Tot_Rept_Positions_Short_All
cftc_gas.sort_values(by = 'Report_Date_as_MM_DD_YYYY', ascending = True, inplace = True )
cftc_gas['rolling_52_min'] = cftc_gas.difference.rolling(52).min()
cftc_gas['rolling_52_max'] = cftc_gas.difference.rolling(52).max()
cftc_gas['Williams_coef'] = round((cftc_gas.difference - cftc_gas['rolling_52_min']) / (cftc_gas['rolling_52_max'] - cftc_gas['rolling_52_min'])*100,2)
cftc_gas['Breeze_coef'] = cftc_gas['Williams_coef'].rolling(window=n_steps).apply(my_fun)
cftc_gas.dropna(inplace = True)

cftc_gas = cftc_gas[['Report_Date_as_MM_DD_YYYY',
                       'Open_Interest_All',
                       'Tot_Rept_Positions_Long_All', 
                       'Tot_Rept_Positions_Short_All',
                       'difference',
                       'rolling_52_min',
                       'rolling_52_max',
                       'Williams_coef',
                       'Breeze_coef']]

## Итоговые датафреймы

In [104]:
print('Exxon min: ', exxon.Date.min())
print('BP min: ', BP.Date.min())
print('shell min: ', shell.Date.min())
print('petrochina min: ', petrochina.Date.min())

Exxon min:  2021-07-02 00:00:00
BP min:  2021-07-02 00:00:00
shell min:  2021-07-02 00:00:00
petrochina min:  2021-07-02 00:00:00


In [105]:
# cftc_gas
exxon_gas = exxon.loc[exxon.Date >= cftc_gas.Report_Date_as_MM_DD_YYYY.min()]
bp_gas = BP.loc[BP.Date >= cftc_gas.Report_Date_as_MM_DD_YYYY.min()]
shell_gas = shell.loc[shell.Date >= cftc_gas.Report_Date_as_MM_DD_YYYY.min()]
petrochina_gas = petrochina.loc[petrochina.Date >= cftc_gas.Report_Date_as_MM_DD_YYYY.min()]

# cftc_oil
exxon_oil = exxon.loc[exxon.Date >= cftc_oil.Report_Date_as_MM_DD_YYYY.min()]
bp_oil = BP.loc[BP.Date >= cftc_oil.Report_Date_as_MM_DD_YYYY.min()]
shell_oil = shell.loc[shell.Date >= cftc_oil.Report_Date_as_MM_DD_YYYY.min()]
petrochina_oil = petrochina.loc[petrochina.Date >= cftc_oil.Report_Date_as_MM_DD_YYYY.min()]

# cftc_gold
newportgold = newportgold.loc[newportgold.Date >= cftc_gold.Report_Date_as_MM_DD_YYYY.min()]
barrickgold = barrickgold.loc[barrickgold.Date >= cftc_gold.Report_Date_as_MM_DD_YYYY.min()]

### Работаем сначала с ГАЗОМ

In [106]:
exxon_gas2 = pd.merge(exxon_gas, cftc_gas[['Williams_coef',
                                          'Breeze_coef',
                                          'Report_Date_as_MM_DD_YYYY',
                                          'Open_Interest_All']],
                                           how = 'left',
                                           left_on = 'Date',
                                           right_on = 'Report_Date_as_MM_DD_YYYY')

exxon_gas2.fillna(method="ffill", inplace = True)
exxon_gas2.drop(columns = 'Report_Date_as_MM_DD_YYYY', inplace = True)


bp_gas2 = pd.merge(bp_gas, cftc_gas[['Williams_coef',
                                    'Breeze_coef',
                                    'Report_Date_as_MM_DD_YYYY',
                                    'Open_Interest_All']],
                                     how = 'left',
                                     left_on = 'Date',
                                     right_on = 'Report_Date_as_MM_DD_YYYY')
bp_gas2.fillna(method="ffill", inplace = True)
bp_gas2.drop(columns = 'Report_Date_as_MM_DD_YYYY', inplace = True)

shell_gas2 = pd.merge(shell_gas, cftc_gas[['Williams_coef',
                                          'Breeze_coef',
                                          'Report_Date_as_MM_DD_YYYY',
                                          'Open_Interest_All']],
                                           how = 'left',
                                           left_on = 'Date',
                                           right_on = 'Report_Date_as_MM_DD_YYYY')
shell_gas2.fillna(method="ffill", inplace = True)
shell_gas2.drop(columns = 'Report_Date_as_MM_DD_YYYY', inplace = True)


petrochina_gas2 = pd.merge(petrochina_gas, cftc_gas[['Williams_coef',
                                                    'Breeze_coef',
                                                    'Report_Date_as_MM_DD_YYYY',
                                                    'Open_Interest_All']],
                                                     how = 'left',
                                                     left_on = 'Date',
                                                     right_on = 'Report_Date_as_MM_DD_YYYY')

petrochina_gas2.fillna(method="ffill", inplace = True)
petrochina_gas2.drop(columns = 'Report_Date_as_MM_DD_YYYY', inplace = True)

exxon_gas2 = pd.merge(exxon_gas2, Gas[['Price',
                                     'Date',
                                     'Volume']],
                                      how = 'left',
                                      on = 'Date')
exxon_gas2.rename(columns = {'Price_x': 'stock_price',
                            'Volume_x': 'stock_volume',
                            'Price_y': 'future_price',
                            'Volume_y': 'future_volume',
                            'Open_Interest_All': 'CFTC_OI_ALL'}, inplace = True)


bp_gas2 = pd.merge(bp_gas2, Gas[['Price',
                                     'Date',
                                     'Volume']],
                                      how = 'left',
                                      on = 'Date')
bp_gas2.rename(columns = {'Price_x': 'stock_price',
                            'Volume_x': 'stock_volume',
                            'Price_y': 'future_price',
                            'Volume_y': 'future_volume',
                            'Open_Interest_All': 'CFTC_OI_ALL'}, inplace = True)


shell_gas2 = pd.merge(shell_gas2, Gas[['Price',
                                     'Date',
                                     'Volume']],
                                      how = 'left',
                                      on = 'Date')
shell_gas2.rename(columns = {'Price_x': 'stock_price',
                            'Volume_x': 'stock_volume',
                            'Price_y': 'future_price',
                            'Volume_y': 'future_volume',
                            'Open_Interest_All': 'CFTC_OI_ALL'}, inplace = True)


petrochina_gas2 = pd.merge(petrochina_gas2, Gas[['Price',
                                     'Date',
                                     'Volume']],
                                      how = 'left',
                                      on = 'Date')
petrochina_gas2.rename(columns = {'Price_x': 'stock_price',
                            'Volume_x': 'stock_volume',
                            'Price_y': 'future_price',
                            'Volume_y': 'future_volume',
                            'Open_Interest_All': 'CFTC_OI_ALL'}, inplace = True)

petrochina_gas2.stock_price = petrochina_gas2.stock_price.astype('float')
shell_gas2.stock_price = shell_gas2.stock_price.astype('float')
bp_gas2.stock_price = bp_gas2.stock_price.astype('float')
exxon_gas2.stock_price = exxon_gas2.stock_price.astype('float')

# Рассчитываем к-т корр Спирмена за 1 год (оконная функция) между стоимостью акции и стоимостью фьючерса
petrochina_gas2['stock_to_future_corr'] = petrochina_gas2['stock_price'].rolling(90).corr(petrochina_gas2['future_price'])
shell_gas2['stock_to_future_corr'] = shell_gas2['stock_price'].rolling(90).corr(shell_gas2['future_price'])
bp_gas2['stock_to_future_corr'] = bp_gas2['stock_price'].rolling(90).corr(bp_gas2['future_price'])
exxon_gas2['stock_to_future_corr'] = exxon_gas2['stock_price'].rolling(90).corr(exxon_gas2['future_price'])

days_gas_total = (exxon.loc[exxon.Date >= cftc_gas.Report_Date_as_MM_DD_YYYY.min()].Date.max() - exxon.loc[exxon.Date >= cftc_gas.Report_Date_as_MM_DD_YYYY.min()].Date.min()).days

In [107]:
max_Will = 70
min_Will = 25
max_Breeze = 30
min_Breeze = -30
corr_corr = 0.7

dict_of_happiness = {'max_Will': max_Will, 
'min_Will': min_Will,
'max_Breeze': max_Breeze,
'min_Breeze': min_Breeze,
'corr_corr': corr_corr}

petrochina_gas = petrochina_gas2 # Перепривязывание переменных нужно, что функция put_call_function могла итерироваться бесконечное число раз
shell_gas = shell_gas2
bp_gas = bp_gas2
exxon_gas = exxon_gas2

exxon_gas.columns.name = 'exxon_gas'
shell_gas.columns.name  = 'shell_gas'
petrochina_gas.columns.name  = 'petrochina_gas'
bp_gas.columns.name  = 'bp_gas'

def put_call_function(df):

    df['long_short_Williams'] = np.where((df.Williams_coef >= max_Will) & (df.stock_to_future_corr >= corr_corr),
                                    str('short'),
                                    np.where((df.Williams_coef <= min_Will) & (df.stock_to_future_corr >= corr_corr), str('long'), "" ))

    df['long_short_Breeze'] = np.where((df.Breeze_coef >= max_Breeze) & (df.stock_to_future_corr >= corr_corr),
                                    str('short'),
                                    np.where((df.Breeze_coef <= min_Breeze) & (df.stock_to_future_corr >= corr_corr), str('long'), "" ))

    df['both'] = np.where(
                         (df['long_short_Breeze'] == "long") 
                          & (df['long_short_Williams'] == df['long_short_Breeze']), 
                          "Both_long",
                          np.where((df['long_short_Breeze'] == "short") 
                          & (df['long_short_Williams'] == df['long_short_Breeze']),
                          "Both_short",
                          "Not_the_same")
                         )
    return df

### Переворачиваем столбец stock_price кверх ногами и находим разницу по столбцам 
# stock_price & shift30 по тем строкам, где прокрасились индексы У, Б.

def get_values_2_4weeks(df):
    df['shift14'] = df.stock_price.shift(-14)
    df['shift21'] = df.stock_price.shift(-21)
    df['shift28'] = df.stock_price.shift(-28)
    df['profit14'] = df.stock_price - df['shift14']
    df['profit21'] = df.stock_price - df['shift21']
    df['profit28'] = df.stock_price - df['shift28']
    df.drop(columns = ['shift14', 'shift21', 'shift28'], inplace = True)
    return df

def dates_grouping(df):    
    df['grp_date'] = df.Date.diff().dt.days.ne(1).cumsum()
    return df

def long_short(df):
    df = df.query('both == "Both_long" | both == "Both_short"') \
               .groupby(['long_short_Breeze', 'grp_date'], as_index = False) \
               .agg({'Date': 'first',
                     'profit14': 'first',
                     'profit21': 'first',
                     'profit28': 'first',
                     'stock_price': 'first'})
    df['month'] = df.Date.dt.month
    df['day'] = (df['Date']).dt.day
    df['ticker'] = df.columns.name
    df['days_rolling'] = abs(df['day'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0]))
    df.drop_duplicates(subset = ['long_short_Breeze', 'month'], keep = 'first', inplace = True)
    df.fillna(11,inplace = True)
    df = df.query('days_rolling > 10')
    return df

gas_list = [exxon_gas,
            shell_gas,
            petrochina_gas,
            bp_gas]

for i in gas_list:
    i = put_call_function(i)
    i = get_values_2_4weeks(i)
    i = dates_grouping(i)

bp_gas = long_short(bp_gas)
shell_gas = long_short(shell_gas)
exxon_gas = long_short(exxon_gas)
petrochina_gas = long_short(petrochina_gas)

gas_list = [exxon_gas,
            shell_gas,
            petrochina_gas,
            bp_gas]

df_gas = pd.DataFrame()
for i in range(len(gas_list)):
    df_gas = pd.concat([df_gas,gas_list[i]], axis = 0, ignore_index=False)
    
df_gas

,long_short_Breeze,grp_date,Date,profit14,profit21,profit28,stock_price,month,day,ticker,days_rolling
0,short,64,2022-09-06,10.97,-4.17,-4.24,94.95,9,6,exxon_gas,11.00
3,short,68,2022-10-03,-13.94,-19.99,-18.58,91.92,10,3,exxon_gas,24.00
0,short,63,2022-09-06,2.25,-0.28,0.71,26.88,9,6,shell_gas,11.00
3,short,67,2022-10-03,-0.38,-2.05,-0.13,26.45,10,3,shell_gas,24.00
0,short,70,2022-10-25,-0.05,-1.85,11.00,42.65,10,25,petrochina_gas,11.00
0,short,64,2022-09-06,3.87,-0.25,0.80,31.13,9,6,bp_gas,11.00
3,short,68,2022-10-03,-1.26,-3.10,-2.85,30.10,10,3,bp_gas,24.00


In [108]:
# Лонг - нет покупок.

In [109]:
# df_gas_short = df_gas.query('long_short_Breeze == "short"')
# df_gas_long = df_gas.query('long_short_Breeze == "long"')

In [110]:
# max_Will = 81
# min_Will = 20
# max_Breeze = 30
# min_Breeze = -30
# corr_corr = 0.8
# df_gas_short = df_gas.query('long_short_Breeze == "short"')
# max_Will = 91
# min_Will = 20
# max_Breeze = 40
# min_Breeze = -40
# corr_corr = 0.6
# df_gas_long = df_gas.query('long_short_Breeze == "long"')
# gas_invest = [df_gas_long, df_gas_short]
# gas_invest = pd.concat(gas_invest,axis = 0)
# gas_invest

# Теперь НЕФТЬ

In [111]:
exxon_oil2 = pd.merge(exxon_oil, cftc_oil[['Williams_coef',
                                          'Breeze_coef',
                                          'Report_Date_as_MM_DD_YYYY',
                                          'Open_Interest_All']],
                                           how = 'left',
                                           left_on = 'Date',
                                           right_on = 'Report_Date_as_MM_DD_YYYY')

exxon_oil2.fillna(method="ffill", inplace = True)
exxon_oil2.drop(columns = 'Report_Date_as_MM_DD_YYYY', inplace = True)

bp_oil2 = pd.merge(bp_oil, cftc_oil[['Williams_coef',
                                    'Breeze_coef',
                                    'Report_Date_as_MM_DD_YYYY',
                                    'Open_Interest_All']],
                                     how = 'left',
                                     left_on = 'Date',
                                     right_on = 'Report_Date_as_MM_DD_YYYY')
bp_oil2.fillna(method="ffill", inplace = True)
bp_oil2.drop(columns = 'Report_Date_as_MM_DD_YYYY', inplace = True)

shell_oil2 = pd.merge(shell_oil, cftc_oil[['Williams_coef',
                                          'Breeze_coef',
                                          'Report_Date_as_MM_DD_YYYY',
                                          'Open_Interest_All']],
                                           how = 'left',
                                           left_on = 'Date',
                                           right_on = 'Report_Date_as_MM_DD_YYYY')
shell_oil2.fillna(method="ffill", inplace = True)
shell_oil2.drop(columns = 'Report_Date_as_MM_DD_YYYY', inplace = True)


petrochina_oil2 = pd.merge(petrochina_oil, cftc_oil[['Williams_coef',
                                                    'Breeze_coef',
                                                    'Report_Date_as_MM_DD_YYYY',
                                                    'Open_Interest_All']],
                                                     how = 'left',
                                                     left_on = 'Date',
                                                     right_on = 'Report_Date_as_MM_DD_YYYY')

petrochina_oil2.fillna(method="ffill", inplace = True)
petrochina_oil2.drop(columns = 'Report_Date_as_MM_DD_YYYY', inplace = True)

exxon_oil2 = pd.merge(exxon_oil2, Oil[['Price',
                                     'Date',
                                     'Volume']],
                                      how = 'left',
                                      on = 'Date')
exxon_oil2.rename(columns = {'Price_x': 'stock_price',
                            'Volume_x': 'stock_volume',
                            'Price_y': 'future_price',
                            'Volume_y': 'future_volume',
                            'Open_Interest_All': 'CFTC_OI_ALL'}, inplace = True)

bp_oil2 = pd.merge(bp_oil2, Oil[['Price',
                                     'Date',
                                     'Volume']],
                                      how = 'left',
                                      on = 'Date')
bp_oil2.rename(columns = {'Price_x': 'stock_price',
                            'Volume_x': 'stock_volume',
                            'Price_y': 'future_price',
                            'Volume_y': 'future_volume',
                            'Open_Interest_All': 'CFTC_OI_ALL'}, inplace = True)

shell_oil2 = pd.merge(shell_oil2, Oil[['Price',
                                     'Date',
                                     'Volume']],
                                      how = 'left',
                                      on = 'Date')
shell_oil2.rename(columns = {'Price_x': 'stock_price',
                            'Volume_x': 'stock_volume',
                            'Price_y': 'future_price',
                            'Volume_y': 'future_volume',
                            'Open_Interest_All': 'CFTC_OI_ALL'}, inplace = True)

petrochina_oil2 = pd.merge(petrochina_oil2, Oil[['Price',
                                     'Date',
                                     'Volume']],
                                      how = 'left',
                                      on = 'Date')
petrochina_oil2.rename(columns = {'Price_x': 'stock_price',
                            'Volume_x': 'stock_volume',
                            'Price_y': 'future_price',
                            'Volume_y': 'future_volume',
                            'Open_Interest_All': 'CFTC_OI_ALL'}, inplace = True)

petrochina_oil2.stock_price = petrochina_oil2.stock_price.astype('float')
shell_oil2.stock_price = shell_oil2.stock_price.astype('float')
bp_oil2.stock_price = bp_oil2.stock_price.astype('float')
exxon_oil2.stock_price = exxon_oil2.stock_price.astype('float')

# Рассчитываем к-т корр Спирмена за 1 год (оконная функция) между стоимостью акции и стоимостью фьючерса
petrochina_oil2['stock_to_future_corr'] = petrochina_oil2['stock_price'].rolling(90).corr(petrochina_oil2['future_price'])
shell_oil2['stock_to_future_corr'] = shell_oil2['stock_price'].rolling(90).corr(shell_oil2['future_price'])
bp_oil2['stock_to_future_corr'] = bp_oil2['stock_price'].rolling(90).corr(bp_oil2['future_price'])
exxon_oil2['stock_to_future_corr'] = exxon_oil2['stock_price'].rolling(90).corr(exxon_oil2['future_price'])

In [112]:
max_Will = 69
min_Will = 20
max_Breeze = 43
min_Breeze = -24
corr_corr = 0.7

dict_of_happiness = {'max_Will': max_Will, 
'min_Will': min_Will,
'max_Breeze': max_Breeze,
'min_Breeze': min_Breeze,
'corr_corr': corr_corr}

petrochina_oil = petrochina_oil2 # Перепривязывание переменных нужно, что функция put_call_function могла итерироваться бесконечное число раз
shell_oil = shell_oil2
bp_oil = bp_oil2
exxon_oil = exxon_oil2

exxon_oil.columns.name = 'exxon_oil'
shell_oil.columns.name  = 'shell_oil'
petrochina_oil.columns.name  = 'petrochina_oil'
bp_oil.columns.name  = 'bp_oil' #Чтобы тикер влез в функции long short

def put_call_function(df):
    df['long_short_Williams'] = np.where((df.Williams_coef >= max_Will) & (df.stock_to_future_corr >= corr_corr),
                                    str('short'),
                                    np.where((df.Williams_coef <= min_Will) & (df.stock_to_future_corr >= corr_corr), str('long'), "" ))

    df['long_short_Breeze'] = np.where((df.Breeze_coef >= max_Breeze) & (df.stock_to_future_corr >= corr_corr),
                                    str('short'),
                                    np.where((df.Breeze_coef <= min_Breeze) & (df.stock_to_future_corr >= corr_corr), str('long'), "" ))

    df['both'] = np.where(
                         (df['long_short_Breeze'] == "long") 
                          & (df['long_short_Williams'] == df['long_short_Breeze']), 
                          "Both_long",
                          np.where((df['long_short_Breeze'] == "short") 
                          & (df['long_short_Williams'] == df['long_short_Breeze']),
                          "Both_short",
                          "Not_the_same")
                         )
    return df

### Переворачиваем столбец stock_price кверх ногами и находим разницу по столбцам 
# stock_price & shift30 по тем строкам, где прокрасились индексы У, Б.

def get_values_2_4weeks(df):
    df['shift14'] = df.stock_price.shift(-14)
    df['shift21'] = df.stock_price.shift(-21)
    df['shift28'] = df.stock_price.shift(-28)
    df['profit14'] = df.stock_price - df['shift14']
    df['profit21'] = df.stock_price - df['shift21']
    df['profit28'] = df.stock_price - df['shift28']
    df.drop(columns = ['shift14', 'shift21', 'shift28'], inplace = True)
    return df

def dates_grouping(df):    
    df['grp_date'] = df.Date.diff().dt.days.ne(1).cumsum()
    return df

def long_short(df):
    df = df.query('both == "Both_long" | both == "Both_short"') \
               .groupby(['long_short_Breeze', 'grp_date'], as_index = False) \
               .agg({'Date': 'first',
                     'profit14': 'first',
                     'profit21': 'first',
                     'profit28': 'first',
                     'stock_price': 'first'})
    df['month'] = df.Date.dt.month
    df['day'] = (df['Date']).dt.day
    df['ticker'] = df.columns.name
    df['days_rolling'] = abs(df['day'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0]))
    df.drop_duplicates(subset = ['long_short_Breeze', 'month'], keep = 'first', inplace = True)
    df.fillna(11,inplace = True)
    df = df.query('days_rolling > 10')
    return df

oil_list = [exxon_oil,
        shell_oil,
        petrochina_oil,
        bp_oil]                

for i in oil_list:
    i = put_call_function(i)
    i = get_values_2_4weeks(i)
    i = dates_grouping(i)

bp_oil = long_short(bp_oil)
shell_oil = long_short(shell_oil)
exxon_oil = long_short(exxon_oil)
petrochina_oil = long_short(petrochina_oil)

oil_list = [exxon_oil,
            shell_oil,
            petrochina_oil,
            bp_oil]

df_oil = pd.DataFrame()
for i in range(len(oil_list)):
    df_oil = pd.concat([df_oil,oil_list[i]], axis = 0, ignore_index=False)
    
df_oil    

,long_short_Breeze,grp_date,Date,profit14,profit21,profit28,stock_price,month,day,ticker,days_rolling
0,long,20,2021-11-08,4.13,3.27,5.69,65.72,11,8,exxon_oil,11.00
2,long,33,2022-02-01,4.37,-0.21,-1.05,80.83,2,1,exxon_oil,14.00
8,short,42,2022-04-04,0.90,-8.54,-5.70,83.16,4,4,exxon_oil,25.00
0,long,19,2021-11-04,-0.18,0.17,0.45,19.53,11,4,shell_oil,11.00
3,long,32,2022-02-01,0.04,-1.21,-0.02,23.38,2,1,shell_oil,14.00
9,short,41,2022-04-04,0.10,-1.69,-2.25,25.24,4,4,shell_oil,25.00
13,short,45,2022-05-02,-2.58,-2.94,-3.15,25.01,5,2,shell_oil,23.00
0,long,20,2021-11-08,4.05,1.05,2.43,48.86,11,8,petrochina_oil,11.00
2,long,33,2022-02-01,0.51,3.59,9.93,52.45,2,1,petrochina_oil,14.00
0,long,20,2021-11-08,1.84,0.52,1.72,28.02,11,8,bp_oil,11.00


In [113]:
# df_oil_short = df_oil.query('long_short_Breeze == "short"')
# df_oil_long = df_oil.query('long_short_Breeze == "long"')
# df_oil_long

,long_short_Breeze,grp_date,Date,profit14,profit21,profit28,stock_price,month,day,ticker,days_rolling
0,long,20,2021-11-08,4.13,3.27,5.69,65.72,11,8,exxon_oil,11.00
2,long,33,2022-02-01,4.37,-0.21,-1.05,80.83,2,1,exxon_oil,14.00
0,long,19,2021-11-04,-0.18,0.17,0.45,19.53,11,4,shell_oil,11.00
3,long,32,2022-02-01,0.04,-1.21,-0.02,23.38,2,1,shell_oil,14.00
0,long,20,2021-11-08,4.05,1.05,2.43,48.86,11,8,petrochina_oil,11.00
2,long,33,2022-02-01,0.51,3.59,9.93,52.45,2,1,petrochina_oil,14.00
0,long,20,2021-11-08,1.84,0.52,1.72,28.02,11,8,bp_oil,11.00
2,long,33,2022-02-01,0.70,3.02,3.90,31.76,2,1,bp_oil,14.00


### Теперь ЗОЛОТО

In [114]:
barrickgold2 = pd.merge(barrickgold, cftc_gold[['Williams_coef',
                                          'Breeze_coef',
                                          'Report_Date_as_MM_DD_YYYY',
                                          'Open_Interest_All']],
                                           how = 'left',
                                           left_on = 'Date',
                                           right_on = 'Report_Date_as_MM_DD_YYYY')
barrickgold2.fillna(method="ffill", inplace = True)
barrickgold2.drop(columns = 'Report_Date_as_MM_DD_YYYY', inplace = True)

newportgold2 = pd.merge(newportgold, cftc_gold[['Williams_coef',
                                                'Breeze_coef',
                                                'Report_Date_as_MM_DD_YYYY',
                                                'Open_Interest_All']],
                                                 how = 'left',
                                                 left_on = 'Date',
                                                 right_on = 'Report_Date_as_MM_DD_YYYY')

newportgold2.fillna(method="ffill", inplace = True)
newportgold2.drop(columns = 'Report_Date_as_MM_DD_YYYY', inplace = True)

barrickgold2 = pd.merge(barrickgold2, gold[['Price',
                                     'Date',
                                     'Volume']],
                                      how = 'left',
                                      on = 'Date')
barrickgold2.rename(columns = {'Price_x': 'stock_price',
                                      'Volume_x': 'stock_volume',
                                      'Price_y': 'future_price',
                                      'Volume_y': 'future_volume',
                                      'Open_Interest_All': 'CFTC_OI_ALL'}, inplace = True)


newportgold2 = pd.merge(newportgold2, gold[['Price',
                                     'Date',
                                     'Volume']],
                                      how = 'left',
                                      on = 'Date')

newportgold2.rename(columns = {'Price_x': 'stock_price',
                            'Volume_x': 'stock_volume',
                            'Price_y': 'future_price',
                            'Volume_y': 'future_volume',
                            'Open_Interest_All': 'CFTC_OI_ALL'}, inplace = True)

newportgold2.stock_price = newportgold2.stock_price.astype('float')
barrickgold2.stock_price = barrickgold2.stock_price.astype('float')

# Рассчитываем к-т корр Спирмена за 1 год (оконная функция) между стоимостью акции и стоимостью фьючерса
newportgold2['stock_to_future_corr'] = newportgold2['stock_price'] \
                                                   .rolling(90).corr(newportgold2['future_price'])
barrickgold2['stock_to_future_corr'] = barrickgold2['stock_price'] \
                                                   .rolling(90).corr(barrickgold2['future_price'])

days_gold_total = (exxon.loc[exxon.Date >= cftc_gold.Report_Date_as_MM_DD_YYYY.min()].Date.max() - exxon.loc[exxon.Date >= cftc_gold.Report_Date_as_MM_DD_YYYY.min()].Date.min()).days

In [119]:

max_Will = 60
min_Will = 50
max_Breeze = 30
min_Breeze = -30
corr_corr = 0.8

dict_of_happiness = {'max_Will': max_Will, 
'min_Will': min_Will,
'max_Breeze': max_Breeze,
'min_Breeze': min_Breeze,
'corr_corr': corr_corr}

newportgold = newportgold2 # Перепривязывание переменных нужно, что функция put_call_function могла итерироваться бесконечное число раз
barrickgold = barrickgold2

barrickgold.columns.name = 'barrickgold'
newportgold.columns.name = 'newportgold'

def put_call_function(df):
    df['long_short_Williams'] = np.where((df.Williams_coef >= max_Will) & (df.stock_to_future_corr >= corr_corr),
                                    str('short'),
                                    np.where((df.Williams_coef <= min_Will) & (df.stock_to_future_corr >= corr_corr), str('long'), "" ))

    df['long_short_Breeze'] = np.where((df.Breeze_coef >= max_Breeze) & (df.stock_to_future_corr >= corr_corr),
                                    str('short'),
                                    np.where((df.Breeze_coef <= min_Breeze) & (df.stock_to_future_corr >= corr_corr), str('long'), "" ))

    df['both'] = np.where(
                         (df['long_short_Breeze'] == "long") 
                          & (df['long_short_Williams'] == df['long_short_Breeze']), 
                          "Both_long",
                          np.where((df['long_short_Breeze'] == "short") 
                          & (df['long_short_Williams'] == df['long_short_Breeze']),
                          "Both_short",
                          "Not_the_same")
                         )
    return df

### Переворачиваем столбец stock_price кверх ногами и находим разницу по столбцам 
# stock_price & shift30 по тем строкам, где прокрасились индексы У, Б.

def get_values_2_4weeks(df):
    df['shift14'] = df.stock_price.shift(-14)
    df['shift21'] = df.stock_price.shift(-21)
    df['shift28'] = df.stock_price.shift(-28)
    df['profit14'] = df.stock_price - df['shift14']
    df['profit21'] = df.stock_price - df['shift21']
    df['profit28'] = df.stock_price - df['shift28']
    df.drop(columns = ['shift14', 'shift21', 'shift28'], inplace = True)
    return df

def dates_grouping(df):    
    df['grp_date'] = df.Date.diff().dt.days.ne(1).cumsum()
    return df

def long_short(df):
    df = df.query('both == "Both_long" | both == "Both_short"') \
               .groupby(['long_short_Breeze', 'grp_date'], as_index = False) \
               .agg({'Date': 'first',
                     'profit14': 'first',
                     'profit21': 'first',
                     'profit28': 'first',
                     'stock_price': 'first'})
    df['month'] = df.Date.dt.month
    df['day'] = (df['Date']).dt.day
    df['ticker'] = df.columns.name
    df['days_rolling'] = abs(df['day'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0]))
    df.drop_duplicates(subset = ['long_short_Breeze', 'month'], keep = 'first', inplace = True)
    df.fillna(11,inplace = True)
    df = df.query('days_rolling > 10')
    return df

gold_list = [barrickgold,
        newportgold]                

for i in gold_list:

    i = put_call_function(i)
    i = get_values_2_4weeks(i)
    i = dates_grouping(i)

barrickgold = long_short(barrickgold)
newportgold = long_short(newportgold)

gold_list = [barrickgold,
        newportgold]

df_gold = pd.DataFrame()
for i in range(len(gold_list)):
    df_gold = pd.concat([df_gold,gold_list[i]], axis = 0, ignore_index=False)
    
df_gold

,long_short_Breeze,grp_date,Date,profit14,profit21,profit28,stock_price,month,day,ticker,days_rolling
0,long,39,2022-03-15,-1.09,-1.81,1.04,23.77,3,15,barrickgold,11.00
3,long,42,2022-04-05,2.09,1.90,3.94,24.46,4,5,barrickgold,23.00
0,long,39,2022-03-15,-7.88,-10.93,0.76,73.37,3,15,newportgold,11.00
3,long,42,2022-04-05,7.73,7.52,14.41,79.86,4,5,newportgold,23.00
